# PAL-flavoured Datatree

The [xarray Datatree](https://xarray-datatree.readthedocs.io) is used as the core data structure for SwarmPAL. You can think of this like a file directory (a tree) which contains an arbitrary number of related xarray datasets. Data can be fetched from different resources (including VirES) and stored in a `Datatree`.

`PalDataItem` provides tools to construct an `xarray.Dataset` from different sources (VirES, HAPI, etc). `create_paldata` helps to construct a `Datatree` from a set of those datasets.

In [ ]:
import datetime as dt

## Fetching data

In [ ]:
from swarmpal.io import create_paldata, PalDataItem

### from VirES API

In [ ]:
# Set of options which are passed to viresclient
data_params = dict(
    collection="SW_OPER_MAGA_LR_1B",
    measurements=["B_NEC"],
    models=["IGRF"],
    start_time="2016-01-01T00:00:00",
    end_time="2016-01-01T03:00:00",
    # start_time=dt.datetime(2016, 1, 1),  # Can use ISO string or datetime
    # end_time=dt.datetime(2016, 1, 1, 3),
    server_url="https://vires.services/ows",
    options=dict(asynchronous=False, show_progress=False),
)
# create_paldata takes an arbitrary number of kwargs
# (these kwargs define the location in the tree and the dataset content)
# This will probably change to make more user-friendly
data = create_paldata(**{"sample/alpha": PalDataItem.from_vires(**data_params)})
print(data)

In [ ]:
# Interactive view of the datatree
data

In [ ]:
# Refer to a branch of the tree like:
data["sample/alpha"]

In [ ]:
# Note that the above is actually a Datatree object
# To get a view of the Dataset:
data["sample/alpha"].ds

## `swarmpal` accessor

The behaviour of the datatree is extended by the addition of an ["accessor"](https://docs.xarray.dev/en/stable/internals/extending-xarray.html) that adds functionality from SwarmPAL under the `.swarmpal` namespace, e.g.:

In [ ]:
# Metadata related to the SwarmPAL framework
data.swarmpal.pal_meta

In [ ]:
data.swarmpal.magnetic_model_varnames

The above properties are constructed from metadata which are stored within the datatree itself:

In [ ]:
data["sample/alpha"].attrs["PAL_meta"]

## Defining and running a `PalProcess`

A process can be defined which will act on datatrees obtained as above. Define processes by subclassing the abstract `PalProcess` class.

In [ ]:
from swarmpal.io import PalProcess

In [ ]:
help(PalProcess)

Here is an example of defining a process. Still subject to change!

Two methods must be set:
- `process_name` identifies the process, and is used to update the `"PAL_meta"` attribute in the datatree when the process is applied.
- `_call` defines the behaviour of the process itself, and should accept the input datatree and return a modified datatree

When a process object is instantiated, the user provides two arguments which are set as properties of the process
- `active_tree (str)` selects which branch of the tree is to be used
- `config (dict)` provides parameters to control the behaviour of the process

In [ ]:
from datatree import DataTree
from xarray import Dataset


class MyProcess(PalProcess):
    """Compute the first differences on a given variable"""

    @property
    def process_name(self):
        return "MyProcess"

    def _call(self, datatree):
        # Identify inputs for algorithm
        dataset = datatree[f"{self.active_tree}/{self.config.get('dataset')}"].ds
        varname = self.config.get("variable")
        target_data = dataset[varname]
        # Apply the algorithm
        output_data = target_data.diff(dim="Timestamp")
        # Write the outputs into a new path in the datatree and return it
        datatree_path_output = f"{self.active_tree}/output"
        datatree[datatree_path_output] = DataTree(
            data=Dataset(
                data_vars={
                    f"d/dt ({varname})": output_data,
                }
            )
        )
        return datatree

The process can now be created with some configuration:

In [ ]:
process = MyProcess(
    active_tree="sample",
    config={"dataset": "alpha", "variable": "B_NEC"},
)

...and there is a tool to apply this process to the datatree:

In [ ]:
data = data.swarmpal.apply(process)
print(data)

The resulting data can be interrogated with the usual tools (in this case we added a new dataset to the tree under `"sample/output"`):

In [ ]:
data["sample/output"].ds["d/dt (B_NEC)"].plot.line(x="Timestamp");

... and the datatree carries with it the metadata about the process which has been applied:

In [ ]:
data.swarmpal.pal_meta

## More tricks with `create_paldata`

### Fetching data from HAPI

Two differences from using VirES:
- Parameters follow the scheme in `hapiclient`  
  Example: http://hapi-server.org/servers/#server=VirES-for-Swarm&dataset=SW_OPER_MAGA_LR_1B&parameters=B_NEC&start=2016-01-01T00:00:00&stop=2016-01-01T03:00:00&return=script&format=python
- The output dataset is not identical to that retrieved from VirES (variables and their content are the same, but less metadata etc)

In [ ]:
data_params = dict(
    server="https://vires.services/hapi",
    dataset="SW_OPER_MAGA_LR_1B",
    parameters="B_NEC",
    start="2016-01-01T00:00:00",
    stop="2016-01-01T03:00:00",
)
data_hapi = create_paldata(alpha_hapi=PalDataItem.from_hapi(**data_params))
print(data_hapi)

### Time padding

A tuple of `timedelta` can be given as an extra parameter. This extends the retrieved time interval, while storing the original time interval in `"analysis_window"` within the `"Pal_meta"` attribute.

In [ ]:
data_params = dict(
    server="https://vires.services/hapi",
    dataset="SW_OPER_MAGA_LR_1B",
    parameters="B_NEC",
    start="2016-01-01T00:00:00",
    stop="2016-01-01T03:00:00",
    pad_times=(dt.timedelta(hours=1), dt.timedelta(hours=1)),
)
data_hapi = create_paldata(alpha_hapi=PalDataItem.from_hapi(**data_params))
print(data_hapi)